In [17]:
# wd = '/mnt/openfact/users/msawinski/factue-task2'
wd = '/Users/marcinsawinski/Documents/GitHub/factue-task2'
import sys, os
os.chdir(wd)

In [18]:
from factue.methods.llm_calls import load_template_list
for x in load_template_list(job='persuasion', step="detect", prompt_version='v001').keys():
    print(f'"{x}"')

"Name_Calling-Labeling"
"Doubt"
"Questioning_the_Reputation"
"Appeal_to_Fear-Prejudice"
"Appeal_to_Values"
"Exaggeration-Minimisation"
"Loaded_Language"
"Repetition"
"new"


In [19]:
import pandas as pd
from pathlib import Path
import os
root = Path("data/llm_output/persuasion")
# Example path template
files = root.rglob("*/*.parquet")

# Read all files and add file path as a column
df_list = []
for f in files:
    df_part = pd.read_parquet(f)
    df_part['source_file'] = f  # add the file path
    df_list.append(df_part)

# Combine into one DataFrame
df = pd.concat(df_list, ignore_index=True)
df['split'] = df.source_file.astype(str).str.split('/',expand=True)[7]

In [20]:
df.head(3)

,filename,start,end,text_lang,text,label_bin,labels_multi,output_id,prompt_name,prompt_version,...,description,think_content,raw,illegal_value,extra_properties,pred,gold,source_file,error,split
0,20240704_URW_BG_30.txt,0,61,BG,ПРЕДСТАВЯНЕ НА ПОЗИЦИЯТА НА РЕПУБЛИКА БЪЛГАРИЯ...,False,[],GPT_4O_MINI/v001/Name_Calling-Labeling,Name_Calling-Labeling,v001,...,[Technique used: No — The text does not contai...,[None],"[{""description"": ""Technique used: No — The tex...",[None],[None],0,0,data/llm_output/persuasion/detect/GPT_4O_MINI/...,NaN,train
1,20240704_URW_BG_30.txt,65,141,BG,"ля, от вносителите, заповядайте, госпожо Васил...",False,[],GPT_4O_MINI/v001/Name_Calling-Labeling,Name_Calling-Labeling,v001,...,[Technique used: No — The text does not contai...,[None],"[{""description"": ""Technique used: No — The tex...",[None],[None],0,0,data/llm_output/persuasion/detect/GPT_4O_MINI/...,NaN,train
2,20240704_URW_BG_30.txt,145,208,BG,"ОРИЯ ВАСИЛЕВА (ВЕЛИЧИЕ): Благодаря Ви, госпожо...",False,[],GPT_4O_MINI/v001/Name_Calling-Labeling,Name_Calling-Labeling,v001,...,[Technique used: No — There are no loaded labe...,[None],"[{""description"": ""Technique used: No — There a...",[None],[None],0,0,data/llm_output/persuasion/detect/GPT_4O_MINI/...,NaN,train


In [21]:
df.pred.value_counts()

pred
 0    85
 1    53
-1     2
Name: count, dtype: int64

In [22]:
df[df.split=='train'].groupby(["prompt_name","text_lang","model_name","split"])['filename'].agg("count").reset_index()

,prompt_name,text_lang,model_name,split,filename
0,Doubt,BG,gpt-4o-mini,train,20
1,Doubt,RU,gpt-4o-mini,train,20
2,Doubt,SI,gpt-4o-mini,train,20
3,Name_Calling-Labeling,BG,gpt-4o-mini,train,20
4,Name_Calling-Labeling,PL,gpt-4o-mini,train,20
5,Name_Calling-Labeling,RU,gpt-4o-mini,train,20
6,Name_Calling-Labeling,SI,gpt-4o-mini,train,20


In [23]:
def normalize_binary(x):
    return 1 if str(x).strip().lower() in {'1', 'true'} else 0

In [24]:
df['source_file'].value_counts().sort_index()
df['gold'] = df['gold'].apply(normalize_binary)
df['pred'] = df['pred'].apply(normalize_binary)
df['split'] = df.source_file.astype(str).str.split('/',expand=True)[7]

In [25]:
df.prompt_name.value_counts()

prompt_name
Name_Calling-Labeling    80
Doubt                    60
Name: count, dtype: int64

In [26]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
# Define a function to compute metrics for a group
# Assume df has columns: prompt_name, text_lang, gold, pred
rows = []

def is_valid_label(x):
    return x in (0, 1)

rows = []

for prompt_name in df['prompt_name'].unique():
    for text_lang in df.loc[df['prompt_name'] == prompt_name, 'text_lang'].unique():
        for model_name in df.loc[(df['prompt_name'] == prompt_name) & (df['text_lang'] == text_lang), 'model_name'].unique():
            group = df[
                (df['prompt_name'] == prompt_name) &
                (df['text_lang'] == text_lang) &
                (df['model_name'] == model_name)
            ]
            if len(group) > 0:
                row = {
                    'prompt_name': prompt_name,
                    'text_lang': text_lang,
                    'model_name': model_name,
                    'accuracy': accuracy_score(group['gold'], group['pred']),
                    'precision': precision_score(group['gold'], group['pred'], zero_division=0),
                    'recall': recall_score(group['gold'], group['pred'], zero_division=0),
                    'f1': f1_score(group['gold'], group['pred'], zero_division=0),
                    'support': len(group)
                }
                rows.append(row)

results = pd.DataFrame(rows)
results[['accuracy', 'precision','recall', 'f1']] = results[['accuracy', 'precision','recall', 'f1']].round(2)
results

,prompt_name,text_lang,model_name,accuracy,precision,recall,f1,support
0,Name_Calling-Labeling,BG,gpt-4o-mini,0.85,0.00,0.0,0.00,20
1,Name_Calling-Labeling,SI,gpt-4o-mini,0.80,0.20,1.0,0.33,20
2,Name_Calling-Labeling,RU,gpt-4o-mini,0.70,0.00,0.0,0.00,20
3,Name_Calling-Labeling,PL,gpt-4o-mini,0.80,0.20,1.0,0.33,20
4,Doubt,BG,gpt-4o-mini,0.65,0.22,1.0,0.36,20
5,Doubt,SI,gpt-4o-mini,0.60,0.53,1.0,0.69,20
6,Doubt,RU,gpt-4o-mini,0.50,0.00,0.0,0.00,20


In [27]:
import seaborn as sns
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
fig, axes = plt.subplots(len(metrics), 1, figsize=(10, 18), sharex=True)

for i, metric in enumerate(metrics):
    ax = axes[i]
    sns.barplot(
        data=results,
        y='technique_id',
        x=metric,
        hue='text_lang',
        ax=ax,
        palette='muted',
        errorbar=None  # fallback for compatibility
    )
    ax.set_title(metric.capitalize())
    ax.set_xlim(0, 1.05)
    ax.set_ylabel('Technique ID')
    ax.set_xlabel('Score')
    ax.legend(title='Text Language', loc='lower right')

plt.tight_layout()
plt.show()

In [ ]:
# # Create a melt of metrics for faceted visualization
# metrics_df = results.melt(
#     id_vars=['technique_id', 'text_lang'],
#     value_vars=['accuracy', 'precision', 'recall', 'f1'],
#     var_name='metric',
#     value_name='score'
# )

# # Plot
# plt.figure(figsize=(12, 6))
# sns.barplot(data=metrics_df, x='technique_id', y='score', hue='text_lang', palette='muted', errorbar=None)
# plt.title('Metrics by Technique and Language')
# plt.ylim(0, 1.05)
# plt.ylabel('Score')
# plt.xlabel('Technique ID')
# plt.legend(title='Text Language')
# plt.grid(axis='y', linestyle='--', alpha=0.7)
# plt.tight_layout()
# plt.show()

# total

In [ ]:
# Metrics
acc = accuracy_score(df['gold'], df['pred'])
precision = precision_score(df['gold'], df['pred'], zero_division=0)
recall = recall_score(df['gold'], df['pred'], average='binary')
f1 = f1_score(df['gold'], df['pred'], average='binary')

print(f"Accuracy: {acc:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1-score: {f1:.3f}")